## Generative Adversarial Networks (GAN)
- A GAN are made of two distinct models, a generator and a discriminator. 
    #### Generator:
    - The job of the generator is to spawn ‘fake’ images that look like the training images

    #### Discriminator:
    - The job of the discriminator is to look at an image and output whether or not it is a real training image or a fake image from the generator

![](assets/GAN_Diagram.png "Generative Adversarial Networks")


### Deep Convultion Genration Adversarial Network (DCGAN):
- A DCGAN is a direct extension of the GAN described above, except that it explicitly uses convolutional and convolutional-transpose layers in the discriminator and generator, respectively

![](assets/DCGAN_Diagram.png "Generative Adversarial Networks")

In [1]:
import torch
import torch.nn as nn

# Set random seed for reproducibility
manualSeed = 49
torch.manual_seed(manualSeed)


In [2]:
dataset_directory = "dataset"
num_worker_threads = 2
batch_size = 128
image_size = 64
num_color_channels = 3
length_latent_vector = 100
num_generator_features= 64
num_discriminator_features = 64
num_epochs = 5
learning_rate = 0.0002
decay_rate = 0.5

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

dataset = datasets.ImageFolder(
    root=dataset_directory,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_worker_threads
)

In [4]:
def custom_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [32]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.layers = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=length_latent_vector,
                out_channels=num_generator_features * 8,
                kernel_size= 4,
                stride= 1,
                padding= 0,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=num_generator_features * 8),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(num_generator_features * 8, num_generator_features * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_generator_features * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_generator_features * 4, num_generator_features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_generator_features * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_generator_features * 2, num_generator_features, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_generator_features),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(num_generator_features, num_color_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.layers(input)

In [33]:
generator = Generator().to(device)
generator.apply(custom_weights_init)

Generator(
  (layers): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )


In [34]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(
                in_channels=num_color_channels,
                out_channels=num_discriminator_features,
                kernel_size= 4,
                stride= 2,
                padding= 1,
                bias=False,
            ),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(num_discriminator_features, num_discriminator_features*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_discriminator_features*2),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(num_discriminator_features*2, num_discriminator_features*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_discriminator_features*4),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(num_discriminator_features*4, num_discriminator_features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_discriminator_features*8),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(num_discriminator_features*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.layers(input)

In [35]:
discriminator = Discriminator()
discriminator.apply(custom_weights_init)

Discriminator(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [40]:
import torch.optim as optim

criterion = nn.BCELoss()

fixed_latent_vector = torch.randn(64, length_latent_vector, 1, 1, device=device)

real_label, fake_label = 1, 0

generator_optimizer = optim.Adam(
    params=generator.parameters(),
    lr=learning_rate,
    betas=(decay_rate, 0.999)
)
discriminator_optimizer = optim.Adam(discriminator.parameters(), learning_rate, (decay_rate, 0.999))

In [41]:
import torchvision.utils as utils

img_list = []
generator_losses = []
discriminator_losses = []
iters = 0

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, start=0):
        # Train discriminator
        discriminator.zero_grad()
        
        real_data = data[0].to(device)
        batch_size = real_data.size(0)
        labels = torch.full(size=(batch_size,), fill_value=real_label, dtype=torch.float, device=device)

        output = discriminator(real_data).view(-1)
        discriminator_real_error = criterion(output, labels)
        discriminator_real_error.backward()

        batch_latent_vectors = torch.randn(batch_size, length_latent_vector, 1, 1, device=device)
        fake_data = generator(batch_latent_vectors)
        labels.fill_(fake_label)

        output = discriminator(fake_data.detach()).view(-1)
        discriminator_fake_error = criterion(output, labels)
        discriminator_fake_error.backward()

        discriminator_error = discriminator_real_error + discriminator_fake_error
        discriminator_optimizer.step()

        # Train Generator
        generator.zero_grad()
        labels.fill_(real_label)

        output = discriminator(fake_data).view(-1)
        generator_error = criterion(output, labels)
        generator_error.backward()

        generator_optimizer.step()

        generator_losses.append(generator_error.item())
        discriminator_losses.append(discriminator_error.item())


        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_Discriminator: %.4f\tLoss_Generator: %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     discriminator_error.item(), generator_error.item()))
        
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = generator(fixed_latent_vector).detach().cpu()
            img_list.append(utils.make_grid(fake, padding=2, normalize=True))

        iters += 1



Starting Training Loop...
[0/5][0/1583]	Loss_D: 0.6365	Loss_G: 8.0958
[0/5][50/1583]	Loss_D: 0.2609	Loss_G: 19.8820
[0/5][100/1583]	Loss_D: 0.5755	Loss_G: 7.8175
[0/5][150/1583]	Loss_D: 0.7333	Loss_G: 5.1656
[0/5][200/1583]	Loss_D: 0.6623	Loss_G: 6.1043
[0/5][250/1583]	Loss_D: 0.3223	Loss_G: 2.3968
[0/5][300/1583]	Loss_D: 0.5939	Loss_G: 2.5630
[0/5][350/1583]	Loss_D: 0.2976	Loss_G: 4.8876
[0/5][400/1583]	Loss_D: 0.4814	Loss_G: 3.5995
[0/5][450/1583]	Loss_D: 0.4013	Loss_G: 3.8118
[0/5][500/1583]	Loss_D: 0.4800	Loss_G: 4.4832
[0/5][550/1583]	Loss_D: 1.1103	Loss_G: 8.0358
[0/5][600/1583]	Loss_D: 0.9404	Loss_G: 6.5775
[0/5][650/1583]	Loss_D: 1.6848	Loss_G: 2.1256
[0/5][700/1583]	Loss_D: 0.9087	Loss_G: 5.4713
[0/5][750/1583]	Loss_D: 0.5027	Loss_G: 3.6556
[0/5][800/1583]	Loss_D: 0.6511	Loss_G: 6.6689
[0/5][850/1583]	Loss_D: 0.3283	Loss_G: 5.5564
[0/5][900/1583]	Loss_D: 0.2936	Loss_G: 3.1724
[0/5][950/1583]	Loss_D: 0.3441	Loss_G: 4.5281
